# Exploring Spark with Pandas


Using pandas examples, convert the analysis to pyspark. This is useful if you discover your data grows too large for your tooling.

The purpose of this notebook is to familiarise yourself you the pyspark API. You are welcome to use the R version of this if you wish. As long as you are able to obtain the correct results. We will be using python in this notebook as it is quite widely used through data science and the community is very large.



#### Firstly, let's get our spark session

In [2]:
pip install pyspark


In [3]:
!lsb_release -a
!apt-get update
# Install java
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
# get spark
VERSION='3.5.0'
!wget https://dlcdn.apache.org/spark/spark-$VERSION/spark-$VERSION-bin-hadoop3.tgz
# decompress spark
!tar xf spark-$VERSION-bin-hadoop3.tgz

# install python package to help with system paths
!pip install -q findspark
# Let Colab know where the java and spark folders are

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/spark-{VERSION}-bin-hadoop3"
# add pyspark to sys.path using findspark
import findspark
findspark.init()

# get a spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

No LSB modules are available.
Distributor ID:	Ubuntu
Description:	Ubuntu 22.04.2 LTS
Release:	22.04
Codename:	jammy
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [515 kB]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:8 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [998 kB]
Hit:9 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,158 kB]
Get:11 http://security.ubuntu.com/ubuntu jammy-security/mult

In [4]:
from pyspark.sql import SparkSession
import pandas as pd
spark = SparkSession.builder.appName('panda-and-spark').getOrCreate()

### Overview


* Joining two dataframes/data sets
* Simple aggregations
* Persisting

#### JOIN: Pandas

We won't use this more in this notebook, but observe how the joins work.

We what happens if you change from the default inner join to outer joins.

In [5]:
customer_raw = [(1, 'bob', 3462543658686),
           (2, 'rob', 9087567565439),
           (3, 'tim', 5436586999467),
           (4, 'tom', 8349756853250)]

customer_cols = ['id', 'name', 'credit_card_number']



orders_raw = [(1, 'ketchup', 'bob', 1.20),
           (2, 'rutabaga', 'bob', 3.35),
           (3, 'fake vegan meat', 'rob', 13.99),
           (4, 'cheesey poofs', 'tim', 3.99),
           (5, 'ice cream', 'tim', 4.95),
           (6, 'protein powder', 'tom', 49.95)]

orders_cols = ['id', 'product_name', 'customer', 'price']

In [6]:
customer_df = pd.DataFrame(customer_raw, columns=customer_cols)
orders_df = pd.DataFrame(orders_raw, columns=orders_cols)

customer_df

joined_df = pd.merge(customer_df, orders_df, how='inner', left_on='name', right_on='customer')
joined_df

## For self study. What happens if (4, 'tom', 8349756853250) in valuesA becomes (4, 'tod', 8349756853250)
## How do the results change?
## More sensibly; what if customers have not made any orders but we still require them in the result set?

,id_x,name,credit_card_number,id_y,product_name,customer,price
0,1,bob,3462543658686,1,ketchup,bob,1.20
1,1,bob,3462543658686,2,rutabaga,bob,3.35
2,2,rob,9087567565439,3,fake vegan meat,rob,13.99
3,3,tim,5436586999467,4,cheesey poofs,tim,3.99
4,3,tim,5436586999467,5,ice cream,tim,4.95
5,4,tom,8349756853250,6,protein powder,tom,49.95


#### JOIN: Spark

In [7]:
customersDF = spark.createDataFrame(customer_raw, customer_cols)

ordersDF = spark.createDataFrame(orders_raw, orders_cols)

# Show tables
customersDF.show()
ordersDF.show()

+---+----+------------------+
| id|name|credit_card_number|
+---+----+------------------+
|  1| bob|     3462543658686|
|  2| rob|     9087567565439|
|  3| tim|     5436586999467|
|  4| tom|     8349756853250|
+---+----+------------------+

+---+---------------+--------+-----+
| id|   product_name|customer|price|
+---+---------------+--------+-----+
|  1|        ketchup|     bob|  1.2|
|  2|       rutabaga|     bob| 3.35|
|  3|fake vegan meat|     rob|13.99|
|  4|  cheesey poofs|     tim| 3.99|
|  5|      ice cream|     tim| 4.95|
|  6| protein powder|     tom|49.95|
+---+---------------+--------+-----+



In [8]:
joinedDF = customersDF.join(ordersDF, customersDF.name == ordersDF.customer)
joinedDF.show()

+---+----+------------------+---+---------------+--------+-----+
| id|name|credit_card_number| id|   product_name|customer|price|
+---+----+------------------+---+---------------+--------+-----+
|  1| bob|     3462543658686|  1|        ketchup|     bob|  1.2|
|  1| bob|     3462543658686|  2|       rutabaga|     bob| 3.35|
|  2| rob|     9087567565439|  3|fake vegan meat|     rob|13.99|
|  3| tim|     5436586999467|  4|  cheesey poofs|     tim| 3.99|
|  3| tim|     5436586999467|  5|      ice cream|     tim| 4.95|
|  4| tom|     8349756853250|  6| protein powder|     tom|49.95|
+---+----+------------------+---+---------------+--------+-----+



## Simple Aggregations

Now let's explore simple aggregations. You will be using these often when doing exploratory work in big data. Remember, the intention here is that you grow familiar with the way the API works, and how to translate inquiries into that API.

> _How much did each person spend?_

In [9]:
joined_df.groupby('name').agg({"price": ["sum"]})

,price
,sum
name,
bob,4.55
rob,13.99
tim,8.94
tom,49.95


In [10]:
import pyspark.sql.functions as f

joinedDF.groupby('name').agg(f.sum('price').alias('total')).show()

+----+-----------------+
|name|            total|
+----+-----------------+
| bob|             4.55|
| rob|            13.99|
| tim|8.940000000000001|
| tom|            49.95|
+----+-----------------+



Let's use bigger data
  * NYC crash data

In [11]:
# save to the filesystem to prevent another load
! curl -o rows.csv https://data.cityofnewyork.us/api/views/h9gi-nx95/rows.csv

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  411M    0  411M    0     0  1360k      0 --:--:--  0:05:09 --:--:-- 1081k


In [12]:
import pandas as pd
nyc_df = pd.read_csv('rows.csv')

<ipython-input-12-a70f860f02a9>:2: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  nyc_df = pd.read_csv('rows.csv')


In [13]:
# number or rows

print(len(nyc_df))

# this is quite large so we will work with a sample while we experiment in pandas as least.

2028253


We'll take a random sample at 20% of the original data

In [14]:
nyc_small = nyc_df.sample(frac=0.2, replace=False, random_state=1)

In [15]:
# we are also going to limit the columns to those we are going to work with

nyc_small = nyc_small[['CRASH DATE', 'CONTRIBUTING FACTOR VEHICLE 1',
                       'BOROUGH', 'VEHICLE TYPE CODE 1',
                       'NUMBER OF PERSONS INJURED']]

In [16]:
nyc_small.head(2)

,CRASH DATE,CONTRIBUTING FACTOR VEHICLE 1,BOROUGH,VEHICLE TYPE CODE 1,NUMBER OF PERSONS INJURED
1394476,01/31/2015,Unspecified,NaN,PASSENGER VEHICLE,0.0
44918,06/21/2021,Failure to Yield Right-of-Way,QUEENS,E-Scooter,1.0


Now, let's create the pyspark dataframe. Now we two frames with the same content
  * nyc_small: pandas
  * sdf_small: pyspark

In [17]:
from pyspark.sql import SQLContext


# there are nan's in the frame with strings, and spark can't 'infer the schema', so we have to help it out
# by replacing them with empty strings and forcing the column to be a string

sdf_small = SQLContext(spark).createDataFrame(nyc_small.fillna('').astype('str'))


# Lets check the schema quickly

print(sdf_small.schema)

/content/spark-3.5.0-bin-hadoop3/python/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


StructType([StructField('CRASH DATE', StringType(), True), StructField('CONTRIBUTING FACTOR VEHICLE 1', StringType(), True), StructField('BOROUGH', StringType(), True), StructField('VEHICLE TYPE CODE 1', StringType(), True), StructField('NUMBER OF PERSONS INJURED', StringType(), True)])


# Questions

Answer the following questions by porting the pandas code to the Spark API



# Question 1


> On what day do most crashes occcur?

In [18]:
# Pandas
nyc_small.groupby('CRASH DATE')['CRASH DATE'].count().sort_values(ascending=False).head(5)

CRASH DATE
01/21/2014    247
11/15/2018    223
01/18/2015    204
05/19/2017    204
03/17/2017    199
Name: CRASH DATE, dtype: int64

In [19]:
### Spark?

sdf_small.groupBy('CRASH DATE').count().orderBy('count', ascending=False).show(5)

sdf_small.count()




+----------+-----+
|CRASH DATE|count|
+----------+-----+
|01/21/2014|  247|
|11/15/2018|  223|
|01/18/2015|  204|
|05/19/2017|  204|
|03/17/2017|  199|
+----------+-----+
only showing top 5 rows



405651

# Question 2

> _Where do most crashes occur?_

In [20]:
nyc_small.groupby('BOROUGH')['BOROUGH'].count().sort_values(ascending=False).head(5)

BOROUGH
BROOKLYN         88719
QUEENS           75103
MANHATTAN        63085
BRONX            41118
STATEN ISLAND    11741
Name: BOROUGH, dtype: int64

In [21]:
## Spark?
sdf_small.groupBy('BOROUGH').count().orderBy('count', ascending=False).show(5)

sdf_small.count()


+---------+------+
|  BOROUGH| count|
+---------+------+
|         |125885|
| BROOKLYN| 88719|
|   QUEENS| 75103|
|MANHATTAN| 63085|
|    BRONX| 41118|
+---------+------+
only showing top 5 rows



405651

 # Question 3

 > What is the most common cause of accident in 'QUEENS'

In [22]:
nyc_small[(nyc_small.BOROUGH == 'QUEENS')]['CONTRIBUTING FACTOR VEHICLE 1'].value_counts()

# you can also use a group by (to avoid the pandas value_counts function)

nyc_small[(nyc_small.BOROUGH == 'QUEENS')].groupby(
    'CONTRIBUTING FACTOR VEHICLE 1'
)['CONTRIBUTING FACTOR VEHICLE 1'].count().sort_values(ascending=False).head(5)

CONTRIBUTING FACTOR VEHICLE 1
Unspecified                       26399
Driver Inattention/Distraction    16122
Failure to Yield Right-of-Way      6236
Backing Unsafely                   3788
Following Too Closely              2748
Name: CONTRIBUTING FACTOR VEHICLE 1, dtype: int64

In [23]:
CONTRIBUTING_FACTOR_VEHICLE_1 = sdf_small.filter(sdf_small["BOROUGH"] == 'QUEENS')
COUNTS = CONTRIBUTING_FACTOR_VEHICLE_1.groupBy('CONTRIBUTING FACTOR VEHICLE 1').count().orderBy('count', ascending = False)

COUNTS.show(5)

+-----------------------------+-----+
|CONTRIBUTING FACTOR VEHICLE 1|count|
+-----------------------------+-----+
|                  Unspecified|26399|
|         Driver Inattentio...|16122|
|         Failure to Yield ...| 6236|
|             Backing Unsafely| 3788|
|         Following Too Clo...| 2748|
+-----------------------------+-----+
only showing top 5 rows



# Question 4

> _What is the average number or injuries for specific cars driving in specific suburbs_


In [24]:
nyc_small.groupby(['VEHICLE TYPE CODE 1', 'BOROUGH'])['NUMBER OF PERSONS INJURED'].mean().sort_values(ascending=False).head(3)

VEHICLE TYPE CODE 1  BOROUGH 
FRONT                BROOKLYN    12.0
PASSE                BRONX       11.0
PC                   BROOKLYN     5.0
Name: NUMBER OF PERSONS INJURED, dtype: float64

In [26]:
## Spark?
grouped_data = sdf_small.groupBy("VEHICLE TYPE CODE 1", "BOROUGH") \
    .agg(f.avg("NUMBER OF PERSONS INJURED").alias("MEAN_INJURED_COUNT")) \
    .orderBy(f.col("MEAN_INJURED_COUNT").desc()) \
    .limit(3)

grouped_data.show()


+-------------------+--------+------------------+
|VEHICLE TYPE CODE 1| BOROUGH|MEAN_INJURED_COUNT|
+-------------------+--------+------------------+
|              FRONT|BROOKLYN|              12.0|
|              PASSE|   BRONX|              11.0|
|                 PC|BROOKLYN|               5.0|
+-------------------+--------+------------------+

